# HOW TO VISUALIZE

Step 1: use the same imports as the original notebook. Also define `tlist` and make sure not to change it during the notebook.
        

## Setup

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from qutip import *

In [ ]:
# Time grid

# If you want to visualize your own, notice that here I used a larger sample rate so that the plot is not jagged
tlist = np.linspace(0,5000,100000)


# Initial state = ground state
psi0 = basis(2, 0)


# No noise
g1 = 0.0 # relaxation (T1)
g2 = 0.0 # dephasing (T2)


# Solver choice
solver = "me"

## Definition of the system

Same as the given notebook definition.

In [ ]:
def qubit_integrate_labframe(omega_0, omega_d, rabi, theta,psi0, solver, phi = 0, g1 = 0, g2 = 0):

    H0 = (omega_0/2) * sigmaz()
    H1 = 2 * rabi * np.sin(theta) * sigmax()
    H2 = 2 * rabi * np.cos(theta) * sigmaz()
    
    def H1_coeff(t, args):
        return np.cos(omega_d*t+phi)
        
    def H2_coeff(t, args):
        return np.cos(omega_d*t+phi)
    
    # collapse operators
    c_ops = []

    if g1 > 0.0:
        c_ops.append(np.sqrt(g1) * sigmam())

    if g2 > 0.0:
        c_ops.append(np.sqrt(g2) * sigmaz())

    e_ops = [sigmax(), sigmay(), sigmaz()]
    
    H = [H0, [H1,H1_coeff],  [H2,H2_coeff]]
    
    if solver == "me": # master equation
        output = mesolve(H, psi0, tlist, c_ops, e_ops)  
    elif solver == "es": # exact schrodinger
        output = essolve(H, psi0, tlist, c_ops, e_ops)  
    elif solver == "mc": # monte carlo
        ntraj = 250
        output = mcsolve(H, psi0, tlist, ntraj, c_ops, e_ops)  
    else:
        raise ValueError("unknown solver")
        
    return output.expect[0], output.expect[1], output.expect[2]

# ADD THIS (visualisation)

In [ ]:
from mpl_toolkits.mplot3d.art3d import Line3D
import matplotlib.animation as anim
from IPython.display import HTML

def bloch_anim(resultset, omega_0, omega_d, theta, lim=100, rate=1, rotatingframe=False, gif_filename=None, gif_fps=30):

    fig = plt.figure()
    b = Bloch(fig=fig)
    b.make_sphere()
    n = lim // rate

    line = Line3D([], [], [], lw=2, color="red")
    arrow = Line3D([0, 0], [0, 0], [0, 0], lw=3, color="blue")

    vectortrace = [np.cos(omega_d*tlist)*np.sin(theta), np.sin(omega_d*tlist), np.cos(omega_d*tlist)*np.cos(theta)]
    
    b.axes.add_artist(line)
    b.axes.add_artist(arrow)

    if rotatingframe:
        resultset = (resultset[0] * np.cos(omega_0 * tlist) + resultset[1] * np.sin(omega_0 * tlist),
                     resultset[0] * -np.sin(omega_0 * tlist) + resultset[1] * np.cos(omega_0 * tlist),
                     resultset[2])
        vectortrace = [np.cos(omega_d*tlist - omega_0*tlist)*np.sin(theta), np.sin(omega_d*tlist - omega_0*tlist), np.cos(omega_d*tlist - omega_0*tlist)*np.cos(theta)]

    else:
        vectortrace = [np.cos(omega_d*tlist)*np.sin(theta), np.sin(omega_d*tlist), np.cos(omega_d*tlist)*np.cos(theta)]

    def draw_frame(fd, *fargs):
        if (fd % 10 == 0):
            print(str(fd)+" ", end="")
        arrow.set_data_3d([0, vectortrace[0][fd*rate]], [0, vectortrace[1][fd*rate]], [0, vectortrace[2][fd*rate]])
        line.set_data_3d(resultset[0][:fd*rate], resultset[1][:fd*rate], resultset[2][:fd*rate])

    ani = anim.FuncAnimation(fig, draw_frame, n)

    if gif_filename is not None:
        writer = anim.PillowWriter(fps=gif_fps, bitrate=1800)
        ani.save(gif_filename, writer=writer)
    
    return HTML(ani.to_jshtml())

### Fundamental resonance (visualization example)

Conditions:
- Drive frequency: omega_d = omega_0
- Inclination angle: theta = pi/2
- Weak driving strength

In [ ]:
omega_0 = 1.0 * np.pi
omega_d = omega_0
rabi   = 0.05 * np.pi # weak driving
theta = 0.5 * np.pi

# If you want to visualize your own, CHANGE THIS LINE from
#   _, _, sz_baseline = qubit_integrate_labframe(
# to 
#   your_var_name = qubit_integrate_labframe(
# (we need to save the whole tuple)
baseline = qubit_integrate_labframe(
    omega_0, omega_d, rabi, theta,
    psi0, solver, g1, g2
)

In [ ]:
plt.figure(figsize=(10,4))

# If you want to visualize your own, CHANGE THIS LINE from
#   plt.plot(tlist, sz_baseline)
# to 
#   plt.plot(tlist, your_var_name[2])
plt.plot(tlist, baseline[2])
plt.title("Fundamental resonance")
plt.xlabel("Time")
plt.ylabel("⟨sigma_z⟩")
plt.ylim(-1.05, 1.05)
plt.show()

In [ ]:
## THIS IS THE VISUALIZATION PART

# `baseline` = your_var_name (the simulation results)
# `omega_0` is the Larmor frequency used to calculate the rotating frame
# `omega_d` is the drive frequency used to draw the blue drive vector
# `theta` is the inclination of the drive vector (though this is a bit hard to see)
bloch_anim(baseline, omega_0, omega_d, theta, 
           # frame count = lim / rate   (if this is too big, you'll get an error saying frames were dropped.
           #                             consider editing the rc parameter like it suggests. )
           lim=500,  # the number of time samples 
           rate=2,   # the number of time samples per frame (this needs to be higher for subharmonic behavior)
           rotatingframe=False, #  rotating (True) or lab (False) frame
           gif_filename="aaaa.gif",   #  set to a string to export to GIF
           gif_fps=25           #  gif framerate
)

### More examples (Sub-harmonic resonances)

We now introduce sub-harmonic resonances. To compare the fundamental resonance with a sub-harmonic resonance, all system parameters were kept identical while only the driving frequency ω_d was changed from ω_0 to approximately ω_0/3.

In [ ]:
theta = 0.5 * np.pi
omega_d = 1.0 * np.pi * 0.3371
rabi   = 0.05 * np.pi

sub_3 = qubit_integrate_labframe(
omega_0, omega_d, rabi, theta,
psi0, solver, g1, g2
)

### Comparison Plot

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(tlist, baseline[2], label="Baseline")
plt.plot(tlist, sub_3[2], label="Sub-harmonic for theta 0.5*pi")


plt.xlabel("Time")
plt.ylabel("⟨simga_z⟩")
plt.ylim(-1.05, 1.05)
plt.legend()
plt.title("Baseline vs sub-harmonic resonance")
plt.show()

In [ ]:
def bloch_anim(resultset, omega_0, omega_d, theta, lim=100, rate=1, rotatingframe=False, gif_filename=None, gif_fps=30):

    fig = plt.figure()
    b = Bloch(fig=fig)
    b.make_sphere()
    n = lim // rate

    line = Line3D([], [], [], lw=2, color="red")
    arrow = Line3D([0, 0], [0, 0], [0, 0], lw=3, color="blue")

    vectortrace = [np.cos(omega_d*tlist)*np.sin(theta), np.sin(omega_d*tlist), np.cos(omega_d*tlist)*np.cos(theta)]
    
    b.axes.add_artist(line)
    b.axes.add_artist(arrow)

    if rotatingframe:
        resultset = (resultset[0] * np.cos(omega_0 * tlist) + resultset[1] * np.sin(omega_0 * tlist),
                     resultset[0] * -np.sin(omega_0 * tlist) + resultset[1] * np.cos(omega_0 * tlist),
                     resultset[2])
        vectortrace = [np.cos(omega_d*tlist - omega_0*tlist)*np.sin(theta), np.sin(omega_d*tlist - omega_0*tlist), np.cos(omega_d*tlist - omega_0*tlist)*np.cos(theta)]

    else:
        vectortrace = [np.cos(omega_d*tlist)*np.sin(theta), np.sin(omega_d*tlist), np.cos(omega_d*tlist)*np.cos(theta)]

    def draw_frame(fd, *fargs):
        if (fd % 10 == 0):
            print(str(fd)+" ", end="")
        arrow.set_data_3d([0, vectortrace[0][fd*rate]], [0, vectortrace[1][fd*rate]], [0, vectortrace[2][fd*rate]])
        line.set_data_3d(resultset[0][:fd*rate], resultset[1][:fd*rate], resultset[2][:fd*rate])

    ani = anim.FuncAnimation(fig, draw_frame, n)

    if gif_filename is not None:
        writer = anim.PillowWriter(fps=gif_fps, bitrate=1800)
        ani.save(gif_filename, writer=writer)
    
    return HTML(ani.to_jshtml())

In [ ]:

theta = 0.5 * np.pi
omega_d = omega_0
rabi   = 0.05 * np.pi

baseline = qubit_integrate_labframe(
omega_0, omega_d, rabi, theta,
psi0, solver, g1, g2
)

In [ ]:
bloch_anim(baseline, omega_0, omega_0, theta, 500, 2, False)

In [ ]:
bloch_anim(baseline, omega_0, omega_0, theta, 500, 2, True)

In [ ]:
theta = 0.5 * np.pi
omega_d = 1.0 * np.pi * 0.3371
rabi   = 0.05 * np.pi

sub_3371 = qubit_integrate_labframe(
omega_0, omega_d, rabi, theta,
psi0, solver, g1, g2
)

In [ ]:
bloch_anim(sub_3371, omega_0, omega_d, theta, 500, 2, True)

In [ ]:
bloch_anim(sub_3371, omega_0, omega_d, theta, 50000, 60, True)

In [ ]:
theta = 0.5 * np.pi
omega_d = 1.0 * np.pi * 1/3
rabi   = 0.05 * np.pi

sub_3 = qubit_integrate_labframe(
omega_0, omega_d, rabi, theta,
psi0, solver, g1, g2
)

In [ ]:
bloch_anim(sub_3, omega_0, omega_d, theta, 5000, 20, True)

In [ ]:
plt.figure(figsize=(12,5))
t2 = np.linspace(0, 20, 10000)

omega_0 = 1.0 * np.pi
omega_d = 1.0 * np.pi * 0.3333

plt.plot(t2, np.cos(omega_d*t2), label="subharmonic / 3")
plt.plot(t2, np.cos(omega_0*t2), label="harmonic")

plt.xlabel("Time (ns)")
plt.ylabel("Driving field")
plt.ylim(-2.05, 2.05)
plt.legend()
plt.title("Baseline vs sub-harmonic resonance (lab frame)")
plt.show()


In [ ]:
plt.figure(figsize=(12,5))
t2 = np.linspace(0, 20, 10000)

omega_0 = 1.0 * np.pi
omega_d = 1.0 * np.pi * 0.3333

plt.plot(t2, np.cos(omega_d*t2 - omega_0*t2), label="subharmonic / 3")
plt.plot(t2, np.cos(omega_0*t2 - omega_0*t2), label="harmonic")

plt.xlabel("Time (ns)")
plt.ylabel("Driving field")
plt.ylim(-2.05, 2.05)
plt.legend()
plt.title("Baseline vs sub-harmonic resonance (rotating frame)")
plt.show()

## Non-transversal field

### Fundamental resonance

In [ ]:
omega_0 = np.pi
omega_d = omega_0

theta = 0.45 * np.pi      # tilted drive
rabi  = 0.05 * np.pi / np.sin(theta)

_, _, sz_baseline_2 = qubit_integrate_labframe(
omega_0, omega_d, rabi, theta,
psi0, solver, g1, g2
)

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(tlist, sz_baseline_2)
plt.title("Baseline (fundamental) resonance")
plt.xlabel("Time")
plt.ylabel("⟨sigma_z⟩")
plt.ylim(-1.05, 1.05)
plt.show()

### Sub-harmonic resonance

In [ ]:
theta = 0.45 * np.pi      # tilted drive
rabi  = 0.05 * np.pi / np.sin(theta)
omega_d = 1.0 * np.pi * 0.508

non_t = qubit_integrate_labframe(
omega_0, omega_d, rabi, theta,
psi0, solver, g1, g2
)

In [ ]:
bloch_anim(non_t, omega_0, omega_d, theta, 5000, 20, False)

In [ ]:
bloch_anim(non_t, omega_0, omega_d, theta, 5000, 20, True)

### Comparison Plot

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(tlist, sz_baseline_2, label="Baseline")
plt.plot(tlist, non_t[3], label="Sub-harmonic for theta 0.45*pi")


plt.xlabel("Time")
plt.ylabel("⟨simga_z⟩")
plt.ylim(-1.05, 1.05)
plt.legend()
plt.title("Baseline vs sub-harmonic resonance")
plt.show()